<a href="https://colab.research.google.com/github/xuziyue/tensorflow-models/blob/main/3_3_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.7.0


In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [3]:
imdb.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [4]:
train_data = imdb['train']
test_data = imdb['test']

In [5]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  # training_sentences.append(s.numpy().decode())
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [6]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [7]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [8]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [9]:
padded

array([[   0,    0,    0, ...,  867,  141,   10],
       [   0,    0,    0, ...,   20,   31,   30],
       [4383, 6109,    2, ...,  550,    5,  735],
       ...,
       [ 861,   36,   11, ...,    8,    8,    2],
       [  52,   11,  217, ...,  251,   94,   42],
       [  84,    5,   30, ...,  131,    7,    6]], dtype=int32)

In [10]:
testing_padded

array([[  11,  772, 1498, ...,   56,   46,  214],
       [  25, 5401,    3, ...,  962,    1,    1],
       [   2,  190,   13, ...,   18, 1163, 1017],
       ...,
       [   6,  104,   86, ...,   60,  514,  544],
       [   0,    0,    0, ...,   53,   71,  222],
       [   0,    0,    0, ...,  448, 8608, 2127]], dtype=int32)

In [11]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [12]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4963 - accuracy: 0.7379 - val_loss: 0.3482 - val_accuracy: 0.8474
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2377 - accuracy: 0.9081 - val_loss: 0.3648 - val_accuracy: 0.8401
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0888 - accuracy: 0.9793 - val_loss: 0.4459 - val_accuracy: 0.8316
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0226 - accuracy: 0.9972 - val_loss: 0.5236 - val_accuracy: 0.8295
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0060 - accuracy: 0.9996 - val_loss: 0.6238 - val_accuracy: 0.8255
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.6397 - val_accuracy: 0.8294
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.7320e-04 - accuracy: 1.0000 - val_loss: 0.6849 - val_accuracy: 0.8319
Ep

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [15]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [16]:
# Visualization https://projector.tensorflow.org/

# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs.tsv')
#   files.download('meta.tsv')

In [17]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
